In [155]:


%load_ext autoreload
%autoreload


import fill_untappd
import yaml, json, sqlite3,  dateparser,os, sys
from pprint import pprint
from airtable import Airtable
import pandas as pd

fill_untappd.run()

import pandas as pd

conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins;", conn)
jsons  = df.json.apply(lambda j: json.loads(j))

venues = jsons.apply(
    lambda x: 
    pd.concat([
        pd.Series(x["venue"]),
        pd.Series(x["user"])[["user_name"]],
        pd.Series(x["beer"]),
        pd.Series(x)[["checkin_id","created_at","rating_score"]]
        ])
        )

venues_info = pd.concat([venues, 
pd.concat([
    venues['contact'].apply(lambda x: pd.Series(x)),
    venues['location'].apply(lambda x: pd.Series(x)),
], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins").astype(int)
venue_last_checkin = venues_info.groupby(["venue_slug","beer_slug"]).created_at.apply(lambda x: x.apply(lambda y:dateparser.parse(y)).max()).rename("last_checkin")


venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts,venue_last_checkin],axis=1))

venues_annotated = venues_annotated.loc[ lambda x: x.venue_url != ""]
venues_annotated = venues_annotated.loc[ venues_annotated.venue_url.notna()]
venues_annotated = venues_annotated.loc[ lambda x: x.venue_city != ""]
venues_annotated = venues_annotated.loc[ venues_annotated.venue_city.notna()]


cans = pd.read_csv("../_data/cans.csv")
cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")
venue_beers_annotated["venue_id"] = venue_beers_annotated["venue_id"].astype(int)
venue_beers_annotated.n_checkins = venue_beers_annotated.n_checkins.astype(int)

venue_beers_annotated.to_csv("../_data/venue_beers.csv")

venues_annotated = venue_beers_annotated.reset_index()[["venue_name","venue_slug","primary_category","contact","venue_id","lat","lng","venue_city" , "venue_address","venue_country","venue_city",	"venue_country",	"venue_state",'venue_url']]
venues_annotated.to_csv("../_data/venues.csv")

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 849140778}
{'count': 0, 'items': []}


In [150]:
fill_untappd.run()

{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 849140778}
{'count': 0, 'items': []}


In [158]:

venues_annotated

,venue_name,venue_slug,primary_category,contact,venue_id,lat,lng,venue_city,venue_address,venue_country,venue_city,venue_country,venue_state,venue_url
0,Dorchester Brewing Company,dorchester-brewing-company,Nightlife Spot,"{'twitter': '@dorchesterbrew', 'venue_url': 'h...",4367121,42.3220,-71.0626,Boston,1250 Massachusetts Ave,United States,Boston,United States,MA,http://www.dorchesterbrewing.com
1,Flatbread Company at Sacco's Bowl Haven,flatbread-company-at-saccos-bowl-haven,Food,"{'twitter': '@flatbreadsville', 'venue_url': '...",12317,42.3957,-71.1237,Somerville,45 Day St,United States,Somerville,United States,MA,www.saccobowl.com
2,Flynn's Irish Pub - Mansfield,flynns-irish-pub,Food,"{'twitter': '@FlynnsIrishPub', 'venue_url': 'h...",1265320,42.0297,-71.2180,Mansfield,219-223 N Main St,United States,Mansfield,United States,MA,http://www.gotoflynns.com/
3,"Danvers, MA",danvers-ma,Outdoors & Recreation,"{'twitter': '', 'venue_url': 'http://www.danve...",95275,42.5751,-70.9301,,,United States,,United States,MA,http://www.danvers.govoffice.com
4,"Danvers, MA",danvers-ma,Outdoors & Recreation,"{'twitter': '', 'venue_url': 'http://www.danve...",95275,42.5751,-70.9301,,,United States,,United States,MA,http://www.danvers.govoffice.com


In [157]:

venues_annotated.venue_url

0    http://www.dorchesterbrewing.com
1                   www.saccobowl.com
2          http://www.gotoflynns.com/
3    http://www.danvers.govoffice.com
4    http://www.danvers.govoffice.com
Name: venue_url, dtype: object

In [151]:

conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins;", conn)
jsons  = df.json.apply(lambda j: json.loads(j))


0     {'checkin_id': 838537272, 'created_at': 'Sat, ...
1     {'checkin_id': 840159586, 'created_at': 'Wed, ...
2     {'checkin_id': 843595254, 'created_at': 'Fri, ...
3     {'checkin_id': 843598886, 'created_at': 'Fri, ...
4     {'checkin_id': 844777755, 'created_at': 'Sun, ...
5     {'checkin_id': 844837932, 'created_at': 'Sun, ...
6     {'checkin_id': 845367612, 'created_at': 'Mon, ...
7     {'checkin_id': 845370113, 'created_at': 'Mon, ...
8     {'checkin_id': 845401472, 'created_at': 'Mon, ...
9     {'checkin_id': 845483235, 'created_at': 'Mon, ...
10    {'checkin_id': 845525446, 'created_at': 'Tue, ...
11    {'checkin_id': 845563920, 'created_at': 'Tue, ...
12    {'checkin_id': 845583146, 'created_at': 'Tue, ...
13    {'checkin_id': 845584020, 'created_at': 'Tue, ...
14    {'checkin_id': 845704440, 'created_at': 'Tue, ...
15    {'checkin_id': 845977174, 'created_at': 'Tue, ...
16    {'checkin_id': 846951051, 'created_at': 'Thu, ...
17    {'checkin_id': 846979203, 'created_at': 'T

In [86]:

venues_annotated

,venue_name,venue_slug,primary_category,contact,venue_id,lat,lng,venue_city,venue_address,venue_country,venue_city,venue_country,venue_state
0,Dorchester Brewing Company,dorchester-brewing-company,Nightlife Spot,"{'twitter': '@dorchesterbrew', 'venue_url': 'h...",4367121,42.3220,-71.0626,Boston,1250 Massachusetts Ave,United States,Boston,United States,MA
1,Flatbread Company at Sacco's Bowl Haven,flatbread-company-at-saccos-bowl-haven,Food,"{'twitter': '@flatbreadsville', 'venue_url': '...",12317,42.3957,-71.1237,Somerville,45 Day St,United States,Somerville,United States,MA
2,High Tor State Park,high-tor-state-park,Outdoors & Recreation,"{'twitter': '', 'venue_url': ''}",2006735,41.1911,-73.9891,New City,415 S Mountain Rd,United States,New City,United States,NY
3,Flynn's Irish Pub - Mansfield,flynns-irish-pub,Food,"{'twitter': '@FlynnsIrishPub', 'venue_url': 'h...",1265320,42.0297,-71.2180,Mansfield,219-223 N Main St,United States,Mansfield,United States,MA
4,Connecticut River,connecticut-river,Outdoors & Recreation,"{'twitter': '', 'venue_url': ''}",1010984,42.2633,-72.6063,South Hadley,,United States,South Hadley,United States,MA


In [147]:


%load_ext autoreload
%autoreload


import fill_untappd
import yaml, json
import os
from pprint import pprint
from airtable import Airtable
import pandas as pd



import pandas as pd
import sqlite3
conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins;", conn)
jsons  = df.json.apply(lambda j: json.loads(j))

venues = jsons.apply(
    lambda x: 
    pd.concat([
        pd.Series(x["venue"]),
        pd.Series(x["user"])[["user_name"]],
        pd.Series(x["beer"]),
        pd.Series(x)[["checkin_id","created_at","rating_score"]]
        ])
        )

venues_info = pd.concat([venues, 
pd.concat([
    venues['contact'].apply(lambda x: pd.Series(x)),
    venues['location'].apply(lambda x: pd.Series(x)),
], axis = 1)],axis=1).set_index(["venue_slug","beer_slug"])

venue_users = venues_info.groupby(["venue_slug","beer_slug"]).user_name.apply(','.join).rename("user_names")
venue_rating = venues_info.groupby(["venue_slug","beer_slug"]).rating_score.sum().rename("total_rating")
venue_counts = venues_info.groupby(["venue_slug","beer_slug"]).size().rename("n_checkins")

venues_info_dedup = venues_info.loc[~venues_info.index.duplicated()]
venues_annotated = venues_info_dedup.join(pd.concat([venue_users, venue_rating, venue_counts],axis=1))

venues_annotated = venues_annotated.loc[ lambda x: x.venue_url != ""]
venues_annotated = venues_annotated.loc[ venues_annotated.venue_url.notna()]

cans = pd.read_csv("../_data/cans.csv")
cans["untappd_id"] = cans.untappd.str.extract('(\d*$)').astype(int)

venue_beers_annotated = venues_annotated.loc[lambda x: x.venue_id.notna()]
venue_beers_annotated = venue_beers_annotated.join(cans.set_index("untappd_id")[["code"]], on ="bid")
venue_beers_annotated = venue_beers_annotated.rename({"code":"can_code"}, axis = "columns")



The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload
{'client_id': 'E8847D1FA347C37248DEC90E1D21568E18FC5152', 'client_secret': '199C86117D183A9B6BCB3220098F34CFF81FD562', 'limit': 100, 'min_id': 849140778}
{'count': 0, 'items': []}


In [9]:
import sqlite3
conn = sqlite3.connect("untappd.db")
df = pd.read_sql_query("select * from checkins limit 100;", conn)